# Optimization, objectives, and constraints (this is hard to keep up to date)

The goal of any unconstrained optimization problem is to find the "best" solution or most desirable input values for a given objective function.
In a constrained optimization problem, there is an additional set of constraints that must be satified for the solution to be of interest.

DESC approaches the ideal MHD fixed-boundary equilibrium problem $\mathbf{F}=0$ as an optimization problem (see Theory docs).
That is $\min_{\mathbf{x} \in \mathcal{R}^n} \mathbf{f}(\mathbf{x})$ subject to a system of linear constraints $\mathbf{A}\mathbf{x}=\mathbf{b}$ where the objective to be minimized is the MHD force balance error $\mathbf{F}=\mathbf{J}\times \mathbf{B} - \nabla p$.

This objective is minimized by evaluating the two components of $\mathbf{F}$, given by $f_{\rho}$ and $f_{\beta}$, on a collocation grid.
The resulting vector of residuals $\mathbf{f} = [f_{\rho},f_{\beta}]$ has length equal to twice the number of grid points (colloaction nodes) since each of $f_{\rho}, f_{\beta}$ are evaluated at every collocation node.

The two components of the force balance residuals map the state vector $\mathbf{x}$ to the values of the residuals at the points given by the state vector: $f \colon \mathbf{x} ↦ f(\mathbf{x})$. 
The state vector being minimized over  $\mathbf{x} = [R_{lmn}, Z_{lmn}, \lambda_{lmn}]$ is the vector of the Fourier-Zernike spectral coefficients used to describe the mapping between the toroidal $(R,\phi,Z)$ coordinates and the computational flux coordinates $(\rho,\theta,\zeta)$.
The state vector has one of the following lengths
- `3*eq.R_basis.num_modes` if a non-stellarator symmetric equilibrium, where the number of basis modes for R and Z are the same
- `eq.R_basis.num_modes + 2 * eq.Z_basis.num_modes` if a stellarator-symmetric equilibrium, where $R$ has $cos(m\theta-n\zeta)$ symmetry, and $Z$ and $\lambda$ have $sin(m\theta-n\zeta)$

## Objectives vs. constraints

A typical task for DESC may involve
- solving for a good equilibrium (minimize force balance errors) given constraints like profiles and boundaries
- optimizing for some criteria on the solved equilibrium

The first task would include `ForceBalance()` as the objective function and constriants which fix profiles and boundaries.
A fixed-boundary equilbrium problem requires the fixed-boundary $R_b(\theta,\zeta),Z_b({\theta,\zeta})$ to be given as a linear constraint during the optimization.
In DESC, additionally a `gauge constraint` on $\lambda$ is applied (to make it periodic), since $\lambda$ is only defined up to an additive multiple of $2\pi$, which constitutes another linear constraint to the problem.

The second task may consider `ForceBalance()` as a constraint, so as to not throw away the work done to find a good equilibrium, and some criteria for better quasisymmetry as an objective.
This allows for searching the configuration space (combinations of parameters that define the state of plasma) for configurations with better quasisymmetry while only considering those that are still good equilibriums.

As demonstrated above, the python object `ForceBalance()` of type `Objective` was used as an objective in the optimization sense in the first task and a constraint in the second task.
There is no seperate `Constraint` type class.
An `Objective` type object is an optimization objective if it is supplied for the `objective` argument to the `optimizer` object.
An `Objective` type object is an optimization constraint if it is supplied for the `constraints` argument to the `optimizer` object.

## Feasible direction formulation

In any case, the task given to DESC is to solve a constrained optimization problem.
DESC deals with constrained optimization problem by using the feasible direction formulation.
See for example [page 3 of this reference](https://www.cs.umd.edu/users/oleary/a607/607constr1hand.pdf).

The geometry of this approach is as follows.
Suppose the objective is to minimize a function $f \colon \mathbb{R}^n \to \mathbb{R}$, subject to a linear system of equations that define the constraints given by $A \mathbf{x} = \mathbf{b}$.
These equations sketch a surface: $\text{image}(A) = S \subset \mathbb{R}^n$ that define the set of feasible points.
That is, any point on this surface satisfies $A \mathbf{x} = \mathbf{b}$.
It is more practical to search for minima to $f$ on this surface than blindy through $\mathbb{R}^n$.

With this approach, the iteration defined by the optimization will only consider vectors that are valid candidates (they satisfy the constraints).
Moreover, by only searching for solutions on this surface, we can reduce the constrained optimization problem
$$\min_{\mathbf{x} ∈ \mathbb{R}^n} f(\mathbf{x}) \; \text{such that} \; A \mathbf{x} = \mathbf{b}$$
to an unconstrained one which can be solved with techniques like Newton iteration and least-squares.
Moreover, each step of the iteration may be a potential solution.
$$\min_{\mathbf{x} ∈ S \subset \mathbb{R}^n} f(\mathbf{x})$$

## Removing linear constraints by factoring
We can limit the search space to the relavant surface by factoring the state vector into a particular component and a homogenous component: $\mathbf{x} = \mathbf{x}_{\text{p}} + \mathbf{x}_{\text{h}}$.
The particular component, $\mathbf{x}_{\text{p}}$, satisfies the constraints $A \mathbf{x}_{\text{p}} = \mathbf{b}$.
Meaning $\mathbf{x}_{\text{p}}$ is a vector that points from the origin to a point on the surface $S = \text{image}(A)$.
The homogenous component, $\mathbf{x}_{\text{h}}$, satisfies $A \mathbf{x}_{\text{h}} = \mathbf{0}$.
Meaning $\mathbf{x}_{\text{h}}$ is a vector that points from some point on $S$ to another point on $S$.
Hence, $\mathbf{x}_p + \mathbf{x}_h$ lies on the surface $S$, or in the image of $A$.
$$A \mathbf{x} = A (\mathbf{x}_{\text{p}} + \mathbf{x}_{\text{h}}) = \mathbf{b}$$

Any $\mathbf{x}_{\text{h}}$ can be written as a linear combination of a nullspace basis of $A$: $\; \mathbf{x}_{\text{h}} = Z \mathbf{y}$.
These are the vectors which parameterize the surface $S$.
With this convention, the state variable is
$$\mathbf{x} = \mathbf{x}_p + \mathbf{Z}\mathbf{y}$$
and the optimization problem becomes an unconstrained search for $\mathbf{y}$ that yields
$$\min_{\mathbf{y} ∈ \mathbb{R}^{n - m} \subset \mathbb{R}^n} f(\mathbf{x}_{\text{p}} + Z \mathbf{y})$$

The length of the vector $\mathbf{y}$ corresponds to the number of linearly independent vectors in the nullspace of $A$, or the number of free (unfixed) parameters.
If $A \in \mathbb{R}^{m \times n}$ with rank $m$, then the length of $\mathbf{y}$ will be $n-m$.
Each component of $\mathbf{y}$ corresponds to some unfixed parameter.
As the optimizer iterates through some trajectory by changing these parameters, the optimizer searches over the surface of feasible solutions.

This method is sometimes summarized as projecting away the constraints because the orthogonal projection of $\mathbf{x} - \mathbf{x}_p$ onto the nullspace of $A$ is $\mathbf{x}_h$.
If $Z$ is additionally constructed to be orthonormal, then it is easy to compute $\mathbf{y}$ from $\mathbf{x}$ and vice versa, a desireable quality to recover the solution to the original optimization problem from the simpler one it was reduced to.
Recall that $Z Z^T$ is the orthogonal projection onto the nullspace of $A$.
We have
$$ \begin{align*}
     \mathbf{x} - \mathbf{x}_p & = \mathbf{x}_h \\
                               & = Z \mathbf{y} \\
     Z Z^T (\mathbf{x} - \mathbf{x}_p) & = Z (Z^T Z) \mathbf{y} \\
                                       & = Z \mathbf{y} \\
                                       & = \mathbf{x}_h
\end{align*} $$
The easy way to compute $\mathbf{y}$, or to "project" the full state vector $\mathbf{x}$ into the reduced optimization vector $\mathbf{y}$, is:
$$Z^T (\mathbf{x} - \mathbf{x}_p) = \mathbf{y}$$

### `factorize_linear_constraints`

In DESC, the process discussed above is done in the `factorize_linear_constraints` function of `desc.objectives.utils`.
This next few paragraphs will walk through the important parts of that code.

A given optimization may have multiple constraints.
The "parallel-arrays" convention is used to group them.
There are dictionaries denoted $A$, $\mathbf{x}$, and $\mathbf{b}$ where the keys are the names of the constraints (`obj.args[0]` is the class name of the objective), and the values are the matrices associated with that constraint.
So the constraint equations associated with a magnetic well constraint could be queried with:
```python
key = "Magnetic Well"
A_key  = A[key]
xp_key = xp[key]
b_key  = b[key]
```

First, we instantiate the dictionaries and create the vectors for each $\mathbf{x}$ of every constraint.
The `dim_x` attribute of an `objective` is the length of the state vector $\mathbf{x}$.
The `dim_f` attribute is the number of objective equations or the rank of $A$ in $A \mathbf{x} = \mathbf{b}$ when the `objective` object is considered a constraint.
```python
# set state vector
args = np.concatenate([obj.args for obj in constraints])
args = np.concatenate((args, objective_args))
# this is all args used by both constraints and objective
args = [arg for arg in arg_order if arg in args]
dimensions = constraints[0].dimensions
dim_x = 0
x_idx = {}
for arg in objective_args:
    x_idx[arg] = np.arange(dim_x, dim_x + dimensions[arg])
    dim_x += dimensions[arg]

A = {}
b = {}
Ainv = {}
xp = jnp.zeros(dim_x)  # particular solution to Ax=b
constraint_args = []  # all args used in constraints
unfixed_args = []  # subset of constraint args for unfixed objectives
```

Then we loop through each constraint and create the matrices as discussed above.
Note that if the target vector is fully specified, that is the length given by `dimensions[obj.target.arg]` equals the total number of available equations, then we know the target vector should be a solution to the contraints $A \mathbf{x} = \mathbf{b}$ and can set $\mathbf{x}_p$ to match the target vector.

Otherwise, the `else` loop is entered, and we need to actually solve for a solution to the system.
Recall, the compute functions of `Objective` objects first compute some quantity (like $A \mathbf{x}$), then they call the method `self._shift_scale` to subtract out the target quantity.
In other words, the function call `obj.compute_scaled(x)` computes the value of a function of $\mathbf{x}$ defined as $A \mathbf{x} - \mathbf{b}$, which we would prefer to be close to zero.
To compute $\mathbf{b}$, we may store the returned value of the function call: `-1 * obj.compute_scaled(x=0)`.

```python
# linear constraint matrices for each objective
for obj in constraints:
    if obj.fixed and obj.dim_f == dimensions[obj.target_arg]:
        # obj.fixed is always true if the objective is linear
        # if all coefficients are fixed the constraint matrices are not needed
        xp = put(xp, x_idx[obj.target_arg], obj.target)
    else:
        unfixed_args.append(arg)
        A_ = obj.derivatives["jac"][arg](jnp.zeros(dimensions[arg]))
        # using obj.compute instead of obj.target to allow for correct scale/weight
        b_ = -obj.compute_scaled(jnp.zeros(obj.dimensions[arg]))
        Ainv_, Z_ = svd_inv_null(A_)
        A[arg] = A_
        b[arg] = b_
        # need to undo scaling here to work with perturbations
        Ainv[arg] = Ainv_ * obj.weight / obj.normalization
```

Then we merge each individual constraint matrix into one block diagonal system.
That way we can return a single optimization problem back to optimizer.
```python
# full A matrix for all unfixed constraints
if len(A):
    unfixed_idx = jnp.concatenate(
        [x_idx[arg] for arg in arg_order if arg in A.keys()]
    )
    A_full = block_diag(*[A[arg] for arg in arg_order if arg in A.keys()])
    b_full = jnp.concatenate([b[arg] for arg in arg_order if arg in b.keys()])
    Ainv_full, Z = svd_inv_null(A_full)
    xp = put(xp, unfixed_idx, Ainv_full @ b_full)
```

The helper function used above, `desc.utils.svd_inv_null(A)`, returns the pseudoinverse, $A^{\dagger}$, and an orthonormal matrix $Z$ with columns that span the nullspace of A.
We then return functions to the optimizer that compute the reduced optimization vector `x_reduced` (labeled $\mathbf{y}$ in the math discussed above) and recover the full state vector.

```python
def project(x):
    """Project a full state vector into the reduced optimization vector."""
    x_reduced = Z.T @ ((x - xp)[unfixed_idx])
    return jnp.atleast_1d(jnp.squeeze(x_reduced))

def recover(x_reduced):
    """Recover the full state vector from the reduced optimization vector."""
    dx = put(jnp.zeros(dim_x), unfixed_idx, Z @ x_reduced)
    return jnp.atleast_1d(jnp.squeeze(xp + dx))
```

It should be clear that the length of `x_reduced` is equal to the number of free (unfixed) parameters.

## Rebuilding objectives
DESC uses a iterative method to solve and optimizize equilibrium.
Sometimes this invovles changing the resolution of an equilibrium via changing the resolution of the `grid` object belonging to that equilibrium.
(Typlically we start from a low resolution equilibrium, and increase later).
This means many quantities need to be recomputed on the newer grid.
In particular the `Objective` objects that include optimization objectives and constraints need to be rebuilt, so that the values they store are computed on the newer grid.
(See the grid tutorial if you are not familiar with the structure in which computed quantities are stored).

# todo below
https://web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf

## `linear_objectives.py`

- when specifying interior surface as the fixboundary constraint, the self A becomes zernike_radial instead of 1?